##### <p> Samuel Wolfe <br> July 26, 2023 <br> MSBA 206 <br> DMBA Case 21.4 Part 2</p>

In [1]:
%matplotlib inline
from pathlib import Path
import pandas as pd
import requests
import io
import matplotlib.pylab as plt
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from dmba import regressionSummary, classificationSummary, liftChart, gainsChart

In [2]:
def readFile(url):
    download = requests.get(url).content
    # Reading the downloaded content and turning it into a pandas dataframe
    df = pd.read_csv(io.StringIO(download.decode('utf-8')))
    return df
def statslist(df):
    dfStats = pd.DataFrame({'Mean' : df.mean(numeric_only=True),
            'SD' : df.std(numeric_only=True),
            'Min' : df.min(),
            'Max' : df.max(),
            'Median' : df.median(numeric_only=True),
            })
    return dfStats

#### Importing dataset

In [28]:
dfVote = readFile("https://raw.githubusercontent.com/wolfesamk/MSBA-206/main/dmba/Voter-Persuasion.csv")
 #do not need these as they are duplicates of values or states as not useable
dropCol = ['VOTER_ID','SET_NO','Partition','MOVED_AD','opposite']
dfVoteDum = pd.get_dummies(dfVote.drop(columns=dropCol), drop_first=True)
predictors = dfVoteDum.columns.tolist()
outcome = 'MOVED_A'

#### I have opted to first use all variables. If this fails I will split the predictors by type. Derived, Commercial Data, Voterfile, Census

#### Split into partitions

In [29]:
dfVote_T = dfVote[dfVote.Partition == 'T'].drop(columns='Partition')
X_train = dfVote_T.drop(columns=outcome)
Y_train = dfVote_T[outcome]
dfVote_V = dfVote[dfVote.Partition == 'V'].drop(columns='Partition')
X_valid = dfVote_V.drop(columns=outcome)
Y_valid = dfVote_V[outcome]